In [1]:
import os
import typing
import numpy as np
import ta as ta_lib
import pandas as pd
import pandas_ta as ta
import matplotlib.pyplot as plt

pd.options.mode.use_inf_as_na = True

import quantstats as qs
qs.extend_pandas()

# filter warnings
import warnings
warnings.filterwarnings('ignore')

### 0. Get Daily OHLCV from YFinance

In [26]:
import yfinance as yf

symbol = 'CCRO3.SA'

df_daily = yf.download(symbol, period='2y', interval='1d').reset_index()

# Drop Adj Close
df_daily.drop(columns=['Adj Close'], inplace=True)

# Renaming Datetime column
df_daily = df_daily.rename(columns={'Datetime': 'Date'})

display(df_daily.tail())

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Volume
489,2022-04-18,13.47,13.87,13.41,13.65,9868600
490,2022-04-19,13.55,13.82,13.48,13.75,7455500
491,2022-04-20,13.62,14.10,13.61,13.75,7486400
492,2022-04-22,13.57,13.57,13.25,13.34,11407100
493,2022-04-25,13.17,13.35,12.95,13.32,7244500


### 0.2 Get Hourly OHLCV from YFinance

In [27]:
df_hourly = yf.download(symbol, period='2y', interval='60m').reset_index()

# Drop Adj Close
df_hourly.drop(columns=['Adj Close'], inplace=True)

# Renaming Datetime column
df_hourly = df_hourly.rename(columns={'Datetime': 'date'})

# Set date column as index
df_hourly.set_index('date', inplace=True)

# Filter hours
hours = df_hourly.index.hour
df_hourly = df_hourly.iloc[((hours >= 11) & (hours < 17))]

df_hourly.index = df_hourly.index.tz_convert(None) - pd.Timedelta(hours=3)

display(df_hourly.tail(10))

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Volume
date,,,,,
2022-04-22 13:00:00,13.36,13.45,13.35,13.41,1783300.0
2022-04-22 14:00:00,13.42,13.43,13.35,13.37,646200.0
2022-04-22 15:00:00,13.37,13.41,13.31,13.32,906000.0
2022-04-22 16:00:00,13.32,13.33,13.25,13.27,857900.0
2022-04-25 11:00:00,13.25,13.26,13.09,13.10,1266600.0
2022-04-25 12:00:00,13.09,13.21,13.09,13.19,680800.0
2022-04-25 13:00:00,13.20,13.35,13.19,13.29,974300.0
2022-04-25 14:00:00,13.29,13.32,13.23,13.27,818700.0
2022-04-25 15:00:00,13.27,13.33,13.26,13.30,105000.0


### 0. Get OHLCV hourly from QFLib

In [71]:
ticker = 'B3SA3'

# Get Adjusted 1-min bars
df = pd.read_parquet(f'../../data/market/ohlcv_minute/{ticker}.parquet')

# Get close values
df = df.close

# Convert index
df.index = df.index.tz_convert(None) - pd.Timedelta(hours=3)

# Filter Trading Hours (10h - 17h)
hours = df.index.hour
df = df.iloc[(hours >= 10) & (hours < 17)]

df.rename(columns={f'{ticker}': 'Close'}, inplace=True)

# Resample 5-min
df_5min = df.groupby(pd.Grouper(freq='5min')).first().dropna()

# Resample daily
df_daily = df.resample('D').last().dropna()


### Feature Engineering

In [29]:
def fix_dataset_inconsistencies(dataframe, fill_value=None):
    """TODO"""
    dataframe = dataframe.replace([-np.inf, np.inf], np.nan)

    # This is done to avoid filling middle holes with backfilling.
    if fill_value is None:
        dataframe.iloc[0,:] = dataframe.apply(lambda column: column.iloc[column.first_valid_index()], axis='index')
    else:
        dataframe.iloc[0,:] = dataframe.iloc[0,:].fillna(fill_value)
        
    return dataframe.fillna(axis='index', method='pad').dropna(axis='columns')

def rsi(price: 'pd.Series[pd.Float64Dtype]', period: float) -> 'pd.Series[pd.Float64Dtype]':
    """TODO"""
    r = price.diff()
    upside = np.minimum(r, 0).abs()
    downside = np.maximum(r, 0).abs()
    rs = upside.ewm(alpha=1 / period).mean() / downside.ewm(alpha=1 / period).mean()
    
    return 100 * (1 - (1 + rs) ** -1)

def macd(price: 'pd.Series[pd.Float64Dtype]', fast: float, slow: float, signal: float) -> 'pd.Series[pd.Float64Dtype]':
    """TODO"""
    fm = price.ewm(span=fast, adjust=False).mean()
    sm = price.ewm(span=slow, adjust=False).mean()
    md = fm - sm
    signal = md - md.ewm(span=signal, adjust=False).mean()
    return signal

def generate_all_default_quantstats_features(data):
    """TODO"""
    
    excluded_indicators = [
        'compare',
        'greeks',
        'information_ratio',
        'omega',
        'r2',
        'r_squared',
        'rolling_greeks',
        'warn'
    ]
    
    indicators_list = [f for f in dir(qs.stats) if f[0] != '_' and f not in excluded_indicators]
    
    df = data.copy()
    df = df.set_index('date')
    df.index = pd.DatetimeIndex(df.index)

    for indicator_name in indicators_list:
        try:
            print(indicator_name)
            indicator = qs.stats.__dict__[indicator_name](df['Close'])
            if isinstance(indicator, pd.Series):
                indicator = indicator.to_frame(name=indicator_name)
                df = pd.concat([df, indicator], axis='columns')
        except (pd.errors.InvalidIndexError, ValueError):
            pass

    df = df.reset_index()
    return df

def generate_features(data):
    """TODO"""
    # Automatically-generated using pandas_ta
    df = data.copy()

    strategies = ['candles', 
                  'cycles', 
                  'momentum', 
                  'overlap', 
                  'performance', 
                  'statistics', 
                  'trend', 
                  'volatility', 
                  'volume']

    df.index = pd.DatetimeIndex(df.index)

    cores = os.cpu_count()
    df.ta.cores = cores

#     for strategy in strategies:
#         df.ta.study(strategy)

    df = df.set_index('Date')

    # Generate all default indicators from ta library
    ta_lib.add_all_ta_features(data, 
                            'Open', 
                            'High', 
                            'Low', 
                            'Close', 
                            'Volume', 
                            fillna=True)
    
    data = data.set_index('Date')

    # Custom indicators
    features = pd.DataFrame.from_dict({
        'prev_open': data['Open'].shift(1),
        'prev_high': data['High'].shift(1),
        'prev_low': data['Low'].shift(1),
        'prev_close': data['Close'].shift(1),
        'prev_volume': data['Volume'].shift(1),
        'vol_5': data['Close'].rolling(window=5).std().abs(),
        'vol_10': data['Close'].rolling(window=10).std().abs(),
        'vol_20': data['Close'].rolling(window=20).std().abs(),
        'vol_30': data['Close'].rolling(window=30).std().abs(),
        'vol_50': data['Close'].rolling(window=50).std().abs(),
        'vol_60': data['Close'].rolling(window=60).std().abs(),
        'vol_100': data['Close'].rolling(window=100).std().abs(),
        'vol_200': data['Close'].rolling(window=200).std().abs(),
        'ma_5': data['Close'].rolling(window=5).mean(),
        'ma_10': data['Close'].rolling(window=10).mean(),
        'ma_20': data['Close'].rolling(window=20).mean(),
        'ma_30': data['Close'].rolling(window=30).mean(),
        'ma_50': data['Close'].rolling(window=50).mean(),
        'ma_60': data['Close'].rolling(window=60).mean(),
        'ma_100': data['Close'].rolling(window=100).mean(),
        'ma_200': data['Close'].rolling(window=200).mean(),
        'ema_5': ta_lib.trend.ema_indicator(data['Close'], window=5, fillna=True),
        'ema_10': ta_lib.trend.ema_indicator(data['Close'], window=10, fillna=True),
        'ema_20': ta_lib.trend.ema_indicator(data['Close'], window=20, fillna=True),
        'ema_60': ta_lib.trend.ema_indicator(data['Close'], window=60, fillna=True),
        'ema_64': ta_lib.trend.ema_indicator(data['Close'], window=64, fillna=True),
        'ema_120': ta_lib.trend.ema_indicator(data['Close'], window=120, fillna=True),
        'lr_open': np.log(data['Open']).diff().fillna(0),
        'lr_high': np.log(data['High']).diff().fillna(0),
        'lr_low': np.log(data['Low']).diff().fillna(0),
        'lr_close': np.log(data['Close']).diff().fillna(0),
        'r_volume': data['Close'].diff().fillna(0),
        'rsi_5': rsi(data['Close'], period=5),
        'rsi_10': rsi(data['Close'], period=10),
        'rsi_100': rsi(data['Close'], period=100),
        'rsi_7': rsi(data['Close'], period=7),
        'rsi_28': rsi(data['Close'], period=28),
        'rsi_6': rsi(data['Close'], period=6),
        'rsi_14': rsi(data['Close'], period=14),
        'rsi_26': rsi(data['Close'], period=24),
        'macd_normal': macd(data['Close'], fast=12, slow=26, signal=9),
        'macd_short': macd(data['Close'], fast=10, slow=50, signal=5),
        'macd_long': macd(data['Close'], fast=200, slow=100, signal=50)
    })

    # Concatenate both manually and automatically generated features
    data = pd.concat([data, features], axis='columns').fillna(method='pad')

    # Remove potential column duplicates
    data = data.loc[:,~data.columns.duplicated()]

    # Revert naming convention
    data = data.rename(columns={'Open': 'open', 
                                'High': 'high', 
                                'Low': 'low', 
                                'Close': 'close', 
                                'Volume': 'volume'})

    # Concatenate both manually and automatically generated features
    data = pd.concat([data, df], axis='columns').fillna(method='pad')

    # Remove potential column duplicates
    data = data.loc[:,~data.columns.duplicated()]

    data = data.reset_index()
    
    # Revert naming convention
    data = data.rename(columns={'Date': 'date'})

    # Generate all default quantstats features
    df_quantstats = generate_all_default_quantstats_features(data)

    # Concatenate both manually and automatically generated features
    data = pd.concat([data, df_quantstats], axis='columns').fillna(method='pad')

    # Remove potential column duplicates
    data = data.loc[:,~data.columns.duplicated()]

    # A lot of indicators generate NaNs at the beginning of DataFrames, so remove them
    data = data.iloc[200:]
    data = data.reset_index(drop=True)

    data = fix_dataset_inconsistencies(data, fill_value=None)
    
    data.set_index('date', inplace=True)
    
    return data

In [41]:
data_daily = generate_features(df_daily)

adjusted_sortino
autocorr_penalty
avg_loss
avg_return
avg_win
best
cagr
calmar
common_sense_ratio
comp
compsum
conditional_value_at_risk
consecutive_losses
consecutive_wins
cpc_index
cvar
distribution
drawdown_details
expected_return
expected_shortfall
exposure
gain_to_pain_ratio
geometric_mean
ghpr
implied_volatility
kelly_criterion
kurtosis
max_drawdown
monthly_returns
outlier_loss_ratio
outlier_win_ratio
outliers
payoff_ratio
pct_rank
profit_factor
profit_ratio
rar
recovery_factor
remove_outliers
risk_of_ruin
risk_return_ratio
rolling_sharpe
rolling_sortino
rolling_volatility
ror
serenity_index
sharpe
skew
smart_sharpe
smart_sortino
sortino
tail_ratio
to_drawdown_series
ulcer_index
ulcer_performance_index
upi
value_at_risk
var
volatility
win_loss_ratio
win_rate
worst


In [42]:
data_daily.head()

,open,high,low,close,volume,volume_adi,volume_obv,volume_cmf,volume_fi,volume_em,...,Volume,compsum,implied_volatility,outliers,pct_rank,remove_outliers,rolling_sharpe,rolling_sortino,rolling_volatility,to_drawdown_series
date,,,,,,,,,,,,,,,,,,,,,
2021-02-18,12.41,12.45,12.12,12.21,9533200,-1.809692e+08,113503900,-0.225457,-1.384352e+05,-0.640395,...,9533200,3.319288e+232,0.430161,14.84,10.833333,12.21,-0.568838,-0.847452,0.383456,-0.222293
2021-02-19,12.20,12.28,12.06,12.17,5256900,-1.809692e+08,108247000,-0.203287,-1.486981e+05,-0.481270,...,5256900,4.371503e+233,0.430161,14.84,7.500000,12.17,-0.487293,-0.728236,0.382589,-0.224841
2021-02-22,11.85,11.85,11.25,11.32,33952100,-2.069992e+08,74294900,-0.278102,-4.250212e+06,-1.095662,...,33952100,5.385691e+234,0.430161,14.84,1.666667,11.32,-0.753596,-1.086247,0.394428,-0.278981
2021-02-23,11.41,11.60,11.28,11.55,10192700,-1.999917e+08,84487600,-0.203495,-3.308135e+06,-0.345346,...,10192700,6.759043e+235,0.430161,14.84,3.333333,11.55,-0.593474,-0.858813,0.395343,-0.264331
2021-02-24,11.61,11.99,11.57,11.65,13937800,-2.086199e+08,98425400,-0.243571,-2.636434e+06,1.024551,...,13937800,8.550189e+236,0.430161,14.84,5.000000,11.65,-0.376157,-0.549560,0.392602,-0.257962


In [43]:
data_daily.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 294 entries, 2021-02-18 to 2022-04-25
Columns: 148 entries, open to to_drawdown_series
dtypes: float64(145), int64(3)
memory usage: 342.2 KB


In [44]:
data_daily.shape

(294, 148)

In [49]:
# Renaming columns
df_hourly = df_hourly.rename(columns={'Open': 'open', 'High': 'high', 'Low': 'low', 'Close': 'close', 'Volume': 'volume'})

In [ ]:
# Daily OHLCV with Technical Indicators
data_daily_hourly = data_daily.resample('H').pad()

# Merge Daily info into Hourly Prices
# TODO: Merge do 'data_daily_hourly' em 'df_hourly'
data_daily_hourly = df_hourly.merge(data_daily_hourly, left_index=True, right_index=True, how='inner')

# Filter 
hour = data_daily_hourly.index.hour
data_daily_hourly = data_daily_hourly.iloc[((hour >= 11) & (hour < 17))]

In [ ]:
data_daily_hourly

---

In [ ]:
fig, ax = plt.subplots(figsize=(20,6))
data_daily_hourly.loc['2021':].close.plot(ax=ax)
ax.set_title(f'{symbol[:5]} Closing Prices')
ax.set_ylabel('Price (R$)')
ax.set_xlabel('Date')
plt.legend()
plt.tight_layout();